## create a connection to the database using the library sqlite3

In [379]:
import pandas as pd
import sqlite3

In [380]:
conn = sqlite3.connect('../ex00/data/checking-logs.sqlite.sqlite copy')

## create a new table datamart in the database by joining the tables pageviews and checker using only one query

In [381]:
query = "SELECT COALESCE(t11.uid,t22.uid) as uid, t11.labname, t11.first_commit_ts, t22.first_view_ts\
        FROM (\
            SELECT labname, uid, timestamp AS first_commit_ts\
            FROM checker\
            WHERE status = 'ready' AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') AND numTrials = 1) t11\
            LEFT JOIN\
            (SELECT * FROM ((SELECT uid, datetime as first_view_ts \
            FROM pageviews\
            WHERE uid LIKE 'user_%') t1\
            INNER JOIN\
            (SELECT uid as user_id, min(datetime) as date \
            from pageviews\
            GROUP BY user_id) t2\
            ON t1.uid = t2.user_id AND t1.first_view_ts == t2.date)) t22\
        ON t11.uid = t22.uid"
df = pd.io.sql.read_sql(query, conn, parse_dates=['first_commit_ts', 'first_view_ts'])
df.to_sql('datamart', conn)

## using Pandas methods, create two dataframes: test and control

In [382]:
df = pd.io.sql.read_sql("select * from datamart", conn, parse_dates=['first_commit_ts', 'first_view_ts'])
test = df.dropna()
control = df[df['first_view_ts'].isnull()]
control['first_view_ts'].values[:] = test['first_view_ts'].mean()
print(test)
print(control)
control.to_sql('control', conn)
test.to_sql('test', conn)

     index      uid   labname            first_commit_ts  \
3        3  user_17  project1 2020-04-18 07:56:45.408648   
4        4  user_30    laba04 2020-04-18 13:36:53.971502   
7        7  user_30   laba04s 2020-04-18 14:51:37.498399   
8        8  user_14    laba04 2020-04-18 15:14:00.312338   
11      11  user_14   laba04s 2020-04-18 22:30:30.247628   
18      18  user_19    laba04 2020-04-20 19:05:01.297780   
19      19  user_25    laba04 2020-04-20 19:16:50.673054   
20      20  user_21    laba04 2020-04-21 17:48:00.487806   
21      21  user_30  project1 2020-04-22 12:36:24.053518   
23      23  user_21   laba04s 2020-04-22 20:09:21.857747   
24      24  user_28    laba04 2020-04-22 21:47:19.707242   
27      27  user_17    laba04 2020-04-23 14:24:29.947554   
28      28   user_3    laba04 2020-04-23 20:29:14.054364   
29      29   user_3   laba04s 2020-04-23 20:40:26.883591   
33      33  user_10    laba04 2020-04-25 08:24:52.696624   
34      34  user_10   laba04s 2020-04-25

## close the connection

In [383]:
conn.close()